In [331]:
import torch
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [332]:
print(f"inputs shape  : {inputs.shape}")

inputs shape  : torch.Size([6, 3])


In [333]:
query = inputs[1]
print(f"query : {query}")
attn_scores_2 = torch.empty(inputs.shape[0])
for i , x_i in enumerate(inputs):
  attn_scores_2[i] = torch.dot(x_i , query)
attn_scores_2

query : tensor([0.5500, 0.8700, 0.6600])


tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])

In [334]:
## normalize attn_scores_2
attn_scores_2_tmp = attn_scores_2 / attn_scores_2.sum()
print(f"attn scores 2 tmp : {attn_scores_2_tmp}")

attn scores 2 tmp : tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])


In [335]:
def softmax_naive(x):
  return torch.exp(x) / torch.exp(x).sum(dim = 0)
print(f"exp : {torch.exp(attn_scores_2)}")
attn_scores_2_naive = softmax_naive(attn_scores_2)
print(f"Attention weights : { attn_scores_2_naive }")
print(f"Sum : { attn_scores_2_naive.sum() }")

exp : tensor([2.5971, 4.4593, 4.3728, 2.3243, 2.0279, 2.9639])
Attention weights : tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum : 1.0


In [336]:
# 请注意，在处理大或小的输入值时，这种简单的软max实现（softmave）可能会遇到数值不稳定问题，如溢出和下流。
# 因此，在实践中，建议使用softmax的PyTorch实现，该实现已经对性能进行了广泛的优化：
attn_weights_2 = torch.softmax(attn_scores_2 , dim = 0)
print("Attention weights : " , attn_weights_2)
print("Sum : " , attn_weights_2.sum())

Attention weights :  tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum :  tensor(1.)


In [337]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i , x_i in enumerate(inputs):
  context_vec_2 += attn_weights_2[i] * x_i 
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


In [338]:
## 3.3.2 Computing attention weights for all input tokens 
attn_scores = inputs @ inputs.T 
attn_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [339]:
attn_weights = torch.softmax(attn_scores , dim = -1) 
attn_weights

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [340]:
all_context_vecs = attn_weights @ inputs 
all_context_vecs

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

## 3.4 implementing self-attention with trainable weights

In [341]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2
d_in

3

In [342]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in ,d_out) , requires_grad = False )
W_key = torch.nn.Parameter(torch.rand(d_in , d_out) , requires_grad = False)
W_value = torch.nn.Parameter(torch.rand(d_in , d_out) , requires_grad = False)

W_query , W_key , W_value

(Parameter containing:
 tensor([[0.2961, 0.5166],
         [0.2517, 0.6886],
         [0.0740, 0.8665]]),
 Parameter containing:
 tensor([[0.1366, 0.1025],
         [0.1841, 0.7264],
         [0.3153, 0.6871]]),
 Parameter containing:
 tensor([[0.0756, 0.1966],
         [0.3164, 0.4017],
         [0.1186, 0.8274]]))

In [343]:
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key 
value_2 = x_2 @W_value
query_2 , key_2 , value_2

(tensor([0.4306, 1.4551]), tensor([0.4433, 1.1419]), tensor([0.3951, 1.0037]))

In [344]:
keys = inputs @ W_key 
values = inputs @ W_value 
print(f"keys.shape {keys.shape} ")
print(f"values.shape {values.shape} ")

keys.shape torch.Size([6, 2]) 
values.shape torch.Size([6, 2]) 


In [345]:
keys_2 = keys[1]
attn_score_22 = query_2.dot(keys_2)
attn_score_22


tensor(1.8524)

In [346]:
attn_scores_2 = query_2 @ keys.T 
attn_scores_2

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])

In [347]:
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k ** 0.5 , dim= - 1)
attn_weights_2

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])

In [348]:
context_vec_2 = attn_weights_2 @ values 
context_vec_2

tensor([0.3061, 0.8210])

## 3.4.2 Implementing a compact self-attention Python class


In [349]:
import torch.nn as nn 
class SelfAttention_v1(nn.Module):
  def __init__(self , d_in , d_out):
    super().__init__() 
    self.W_query = nn.Parameter(torch.rand(d_in , d_out))
    self.W_key = nn.Parameter(torch.rand(d_in , d_out))
    self.W_value = nn.Parameter(torch.rand(d_in , d_out))
  def forward(self, x):
    keys = x @ self.W_key
    queries = x @ self.W_query 
    values = x @ self.W_value
    attn_scores  = queries @ keys.T 
    attn_weights = torch.softmax(attn_scores / (keys.shape[-1] ** 0.5) , dim = -1)
    context_vec = attn_weights @ values 
    return context_vec
    

In [350]:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)
